# Tutorial 2: Microstrip Patch Antenna Design

This notebook demonstrates designing and simulating a microstrip patch antenna for 2.4 GHz WiFi.

**Learning objectives:**
- Design a patch antenna for a target frequency
- Configure probe feed for impedance matching
- Compute input impedance and return loss
- Generate radiation patterns
- Analyze bandwidth

In [ ]:
import sys
sys.path.insert(0, '../build/python')
sys.path.insert(0, '../python')

from vectorem.designs import PatchAntennaDesign
import vectorem.plots as vp
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

## 1. Design Parameters

Target: 2.4 GHz patch antenna on FR-4 substrate

In [ ]:
# Target frequency
f_target = 2.4e9  # 2.4 GHz (WiFi)

# FR-4 substrate
eps_r = 4.4       # Relative permittivity
tan_d = 0.02      # Loss tangent
h = 1.6e-3        # Thickness: 1.6 mm

# Initial dimension estimate
c0 = 299792458.0
eps_eff = (eps_r + 1) / 2
lambda_eff = c0 / (f_target * np.sqrt(eps_eff))
L_est = lambda_eff / 2

print(f"Target frequency: {f_target/1e9:.2f} GHz")
print(f"Substrate: εr={eps_r}, h={h*1000:.1f}mm")
print(f"Estimated patch length: {L_est*1000:.1f} mm")

## 2. Create Patch Antenna

In [ ]:
# Create patch antenna
patch = PatchAntennaDesign(
    patch_length=29e-3,      # 29 mm
    patch_width=38e-3,       # 38 mm
    substrate_height=h,
    substrate_eps_r=eps_r,
    substrate_tan_d=tan_d,
)

print(f"Patch: {patch.patch_length*1000:.1f} x {patch.patch_width*1000:.1f} mm")
print(f"Estimated resonant frequency: {patch.estimated_resonant_frequency/1e9:.3f} GHz")

## 3. Configure Feed

Using coaxial probe feed positioned for ~50Ω match:

In [ ]:
# Configure probe feed
# Position offset from patch center (y-direction is resonant dimension)
patch.set_probe_feed(
    x_offset=0,          # Centered in width
    y_offset=-5e-3,      # 5mm from center toward edge
    probe_radius=0.5e-3  # 0.5mm radius
)

print(f"Feed position: x=0, y=-5mm from center")

## 4. Generate Mesh

In [ ]:
# Generate mesh (15 elements per wavelength for antennas)
patch.generate_mesh(density=15)

print(f"Mesh generated: {len(patch.mesh.elements)} elements")

## 5. Input Impedance

In [ ]:
# Compute input impedance at design frequency
Z_in = patch.input_impedance(2.4e9)

print(f"Input impedance at 2.4 GHz:")
print(f"  Z_in = {Z_in.real:.1f} + j{Z_in.imag:.1f} Ω")

# Reflection coefficient for 50Ω system
Z0 = 50
Gamma = (Z_in - Z0) / (Z_in + Z0)
print(f"\nFor 50Ω reference:")
print(f"  |Γ| = {abs(Gamma):.4f}")
print(f"  Return loss = {20*np.log10(abs(Gamma)+1e-10):.1f} dB")
print(f"  VSWR = {(1+abs(Gamma))/(1-abs(Gamma)):.2f}")

## 6. Frequency Sweep

In [ ]:
# Frequency sweep
freqs, Z_in_arr, S11_arr = patch.frequency_sweep(
    f_start=2.0e9,
    f_stop=2.8e9,
    n_points=41,
    z0=50.0,
    verbose=True
)

# Find resonance
rl_db = 20 * np.log10(np.abs(S11_arr) + 1e-10)
idx_min = np.argmin(rl_db)
f_res = freqs[idx_min]

print(f"\nResonant frequency: {f_res/1e9:.3f} GHz")
print(f"Minimum return loss: {rl_db[idx_min]:.1f} dB")

## 7. Plot Return Loss and Impedance

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 8))

# Return loss
axes[0].plot(freqs/1e9, rl_db, 'b-', linewidth=2)
axes[0].axhline(y=-10, color='r', linestyle='--', label='-10 dB (VSWR<2)')
axes[0].axvline(x=f_res/1e9, color='g', linestyle=':', label=f'Resonance: {f_res/1e9:.3f} GHz')
axes[0].set_xlabel('Frequency (GHz)')
axes[0].set_ylabel('Return Loss (dB)')
axes[0].set_title('Patch Antenna Return Loss')
axes[0].legend()
axes[0].grid(True)
axes[0].set_ylim(-30, 0)

# Input impedance
axes[1].plot(freqs/1e9, Z_in_arr.real, 'b-', linewidth=2, label='Re(Z)')
axes[1].plot(freqs/1e9, Z_in_arr.imag, 'r--', linewidth=2, label='Im(Z)')
axes[1].axhline(y=50, color='g', linestyle=':', label='50Ω')
axes[1].axhline(y=0, color='k', linestyle='-', alpha=0.3)
axes[1].set_xlabel('Frequency (GHz)')
axes[1].set_ylabel('Impedance (Ω)')
axes[1].set_title('Input Impedance')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

## 8. Bandwidth Analysis

In [ ]:
# Compute bandwidth for VSWR < 2 (RL < -10 dB)
f_low, f_high, bw_percent = patch.bandwidth(vswr_threshold=2.0, z0=50.0)

print(f"Bandwidth (VSWR < 2.0):")
print(f"  Lower: {f_low/1e9:.3f} GHz")
print(f"  Upper: {f_high/1e9:.3f} GHz")
print(f"  Bandwidth: {bw_percent:.1f}%")
print(f"  Bandwidth: {(f_high-f_low)/1e6:.1f} MHz")

## 9. Radiation Pattern

In [ ]:
# Compute radiation pattern at resonance
pattern = patch.radiation_pattern(f_res, n_theta=91, n_phi=73)

# Compute directivity
D = patch.directivity(f_res)
D_dBi = 10 * np.log10(D)

print(f"Radiation characteristics at {f_res/1e9:.3f} GHz:")
print(f"  Directivity: {D_dBi:.1f} dBi")

In [ ]:
# Plot E-plane and H-plane cuts
vp.plot_pattern_3d_cuts(
    pattern,
    title=f"Patch Antenna at {f_res/1e9:.2f} GHz",
    min_db=-30
)

## 10. Export Results

In [ ]:
# Export S11 to Touchstone
patch.export_touchstone("patch_antenna_2p4ghz", freqs, S11_arr)
print("Saved: patch_antenna_2p4ghz.s1p")

# Export pattern to CSV
patch.export_pattern_csv("patch_pattern.csv", f_res)
print("Saved: patch_pattern.csv")

## 11. Feed Position Optimization

Let's see how feed position affects impedance:

In [ ]:
# Parametric study: vary feed position
feed_positions = np.linspace(-10e-3, -3e-3, 8)
results = []

for y_pos in feed_positions:
    # Create new patch
    p = PatchAntennaDesign(
        patch_length=29e-3,
        patch_width=38e-3,
        substrate_height=1.6e-3,
        substrate_eps_r=4.4,
    )
    p.set_probe_feed(x_offset=0, y_offset=y_pos)
    p.generate_mesh(density=12)  # Faster mesh for parametric
    
    Z = p.input_impedance(2.4e9)
    rl = p.return_loss(2.4e9)
    
    results.append({
        'y_pos': y_pos * 1000,
        'R': Z.real,
        'X': Z.imag,
        'RL': rl
    })
    print(f"y = {y_pos*1000:5.1f} mm: Z = {Z.real:5.0f}+j{Z.imag:5.0f} Ω, RL = {rl:5.1f} dB")

In [ ]:
# Plot feed position vs impedance
y_pos = [r['y_pos'] for r in results]
R = [r['R'] for r in results]
X = [r['X'] for r in results]
RL = [r['RL'] for r in results]

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Impedance vs position
axes[0].plot(y_pos, R, 'b-o', label='Re(Z)')
axes[0].plot(y_pos, X, 'r--s', label='Im(Z)')
axes[0].axhline(y=50, color='g', linestyle=':', label='50Ω')
axes[0].axhline(y=0, color='k', linestyle='-', alpha=0.3)
axes[0].set_xlabel('Feed Y-position (mm)')
axes[0].set_ylabel('Impedance (Ω)')
axes[0].set_title('Input Impedance vs Feed Position')
axes[0].legend()
axes[0].grid(True)

# Return loss vs position
axes[1].plot(y_pos, RL, 'b-o')
axes[1].axhline(y=-10, color='r', linestyle='--', label='-10 dB')
axes[1].set_xlabel('Feed Y-position (mm)')
axes[1].set_ylabel('Return Loss (dB)')
axes[1].set_title('Return Loss vs Feed Position')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

# Find optimal position
best_idx = np.argmin([abs(r['R']-50) + abs(r['X']) for r in results])
print(f"\nBest match at y = {results[best_idx]['y_pos']:.1f} mm")
print(f"  Z = {results[best_idx]['R']:.0f}+j{results[best_idx]['X']:.0f} Ω")

## Summary

In this tutorial we:

1. Designed a 2.4 GHz patch antenna on FR-4 substrate
2. Configured probe feed for impedance matching
3. Computed input impedance and return loss
4. Found resonant frequency and bandwidth
5. Generated radiation pattern and directivity
6. Performed parametric study on feed position

**Key results:**
- Resonant frequency: ~2.4 GHz
- Bandwidth (VSWR<2): ~3-5%
- Directivity: ~6-8 dBi

**Next steps:**
- [Tutorial 3: Unit Cell](03_unit_cell.ipynb)
- [Tutorial 4: Phased Array](04_phased_array.ipynb)